<a href="https://colab.research.google.com/github/amitshmidov/geometric_learning_project/blob/main/geometric_learning_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
!pip install pyvista
!pip install trimesh

     |████████████████████████████████| 1.2MB 6.7MB/s 
     |████████████████████████████████| 103.4MB 49kB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 153kB 41.3MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-cp37-none-any.whl size=59374 sha256=031ce72a886044e9114512a098d2cea367ef8a52015daa0f740081b66873585a
  Stored in directory: /root/.cache/pip/wheels/3d/3c/84/28d36677f3c760c048bd02b5a547ea0c4027770cc9cdb9af1e
Successfully built transforms3d
     |████████████████████████████████| 634kB 6.5MB/s 


In [2]:
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
import pyvista as pv
import matplotlib.pyplot as plt
import os
import trimesh
from tqdm.notebook import tqdm
from trimesh.sample import sample_surface_even

from typing import Tuple
from google.colab import drive
drive.mount('/content/drive')

TensorFlow 1.x selected.
Mounted at /content/drive


# Utils

In [3]:
def _variable_on_cpu(name, shape, initializer, use_fp16=False):
  """Helper to create a Variable stored on CPU memory.
  Args:
    name: name of the variable
    shape: list of ints
    initializer: initializer for Variable
  Returns:
    Variable Tensor
  """
  with tf.device('/cpu:0'):
    dtype = tf.float16 if use_fp16 else tf.float32
    var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
  return var


def _variable_with_weight_decay(name, shape, stddev, wd, use_xavier=True):
  """Helper to create an initialized Variable with weight decay.

  Note that the Variable is initialized with a truncated normal distribution.
  A weight decay is added only if one is specified.

  Args:
    name: name of the variable
    shape: list of ints
    stddev: standard deviation of a truncated Gaussian
    wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
    use_xavier: bool, whether to use xavier initializer

  Returns:
    Variable Tensor
  """
  if use_xavier:
    initializer = tf.contrib.layers.xavier_initializer()
  else:
    initializer = tf.truncated_normal_initializer(stddev=stddev)
  var = _variable_on_cpu(name, shape, initializer)
  if wd is not None:
    weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
    tf.add_to_collection('losses', weight_decay)
  return var


def conv1d(inputs,
          num_output_channels,
          kernel_size,
          scope,
          stride=1,
          padding='SAME',
          use_xavier=True,
          stddev=1e-3,
          weight_decay=0.0,
          activation_fn=tf.nn.relu,
          bn=False,
          bn_decay=None,
          is_training=None):
  """ 1D convolution with non-linear operation.

  Args:
    inputs: 3-D tensor variable BxLxC
    num_output_channels: int
    kernel_size: int
    scope: string
    stride: int
    padding: 'SAME' or 'VALID'
    use_xavier: bool, use xavier_initializer if true
    stddev: float, stddev for truncated_normal init
    weight_decay: float
    activation_fn: function
    bn: bool, whether to use batch norm
    bn_decay: float or float tensor variable in [0,1]
    is_training: bool Tensor variable

  Returns:
    Variable tensor
  """
  with tf.variable_scope(scope) as sc:
    num_in_channels = inputs.get_shape()[-1].value
    kernel_shape = [kernel_size,
                    num_in_channels, num_output_channels]
    kernel = _variable_with_weight_decay('weights',
                                        shape=kernel_shape,
                                        use_xavier=use_xavier,
                                        stddev=stddev,
                                        wd=weight_decay)
    outputs = tf.nn.conv1d(inputs, kernel,
                          stride=stride,
                          padding=padding)
    biases = _variable_on_cpu('biases', [num_output_channels],
                              tf.constant_initializer(0.0))
    outputs = tf.nn.bias_add(outputs, biases)

    if bn:
      outputs = batch_norm_for_conv1d(outputs, is_training,
                                      bn_decay=bn_decay, scope='bn')

    if activation_fn is not None:
      outputs = activation_fn(outputs)
    return outputs


def conv2d(inputs,
          num_output_channels,
          kernel_size,
          scope,
          stride=[1, 1],
          padding='SAME',
          use_xavier=True,
          stddev=1e-3,
          weight_decay=0.0,
          activation_fn=tf.nn.relu,
          bn=False,
          bn_decay=None,
          is_training=None):
  """ 2D convolution with non-linear operation.

  Args:
    inputs: 4-D tensor variable BxHxWxC
    num_output_channels: int
    kernel_size: a list of 2 ints
    scope: string
    stride: a list of 2 ints
    padding: 'SAME' or 'VALID'
    use_xavier: bool, use xavier_initializer if true
    stddev: float, stddev for truncated_normal init
    weight_decay: float
    activation_fn: function
    bn: bool, whether to use batch norm
    bn_decay: float or float tensor variable in [0,1]
    is_training: bool Tensor variable

  Returns:
    Variable tensor
  """
  
  with tf.variable_scope(scope) as sc:
      kernel_h, kernel_w = kernel_size
      num_in_channels = inputs.get_shape()[-1].value
      kernel_shape = [kernel_h, kernel_w,
                      num_in_channels, num_output_channels]
      kernel = _variable_with_weight_decay('weights',
                                          shape=kernel_shape,
                                          use_xavier=use_xavier,
                                          stddev=stddev,
                                          wd=weight_decay)
      stride_h, stride_w = stride
      outputs = tf.nn.conv2d(inputs, kernel,
                            [1, stride_h, stride_w, 1],
                            padding=padding)
      biases = _variable_on_cpu('biases', [num_output_channels],
                                tf.constant_initializer(0.0))
      outputs = tf.nn.bias_add(outputs, biases)

      if bn:
        outputs = batch_norm_for_conv2d(outputs, is_training,
                                        bn_decay=bn_decay, scope='bn')

      if activation_fn is not None:
        outputs = activation_fn(outputs)
      return outputs


def conv2d_transpose(inputs,
                    num_output_channels,
                    kernel_size,
                    scope,
                    stride=[1, 1],
                    padding='SAME',
                    use_xavier=True,
                    stddev=1e-3,
                    weight_decay=0.0,
                    activation_fn=tf.nn.relu,
                    bn=False,
                    bn_decay=None,
                    is_training=None):
  """ 2D convolution transpose with non-linear operation.

  Args:
    inputs: 4-D tensor variable BxHxWxC
    num_output_channels: int
    kernel_size: a list of 2 ints
    scope: string
    stride: a list of 2 ints
    padding: 'SAME' or 'VALID'
    use_xavier: bool, use xavier_initializer if true
    stddev: float, stddev for truncated_normal init
    weight_decay: float
    activation_fn: function
    bn: bool, whether to use batch norm
    bn_decay: float or float tensor variable in [0,1]
    is_training: bool Tensor variable

  Returns:
    Variable tensor

  Note: conv2d(conv2d_transpose(a, num_out, ksize, stride), a.shape[-1], ksize, stride) == a
  """
  with tf.variable_scope(scope) as sc:
      kernel_h, kernel_w = kernel_size
      num_in_channels = inputs.get_shape()[-1].value
      kernel_shape = [kernel_h, kernel_w,
                      num_output_channels, num_in_channels] # reversed to conv2d
      kernel = _variable_with_weight_decay('weights',
                                          shape=kernel_shape,
                                          use_xavier=use_xavier,
                                          stddev=stddev,
                                          wd=weight_decay)
      stride_h, stride_w = stride
      
      # from slim.convolution2d_transpose
      def get_deconv_dim(dim_size, stride_size, kernel_size, padding):
          dim_size *= stride_size

          if padding == 'VALID' and dim_size is not None:
            dim_size += max(kernel_size - stride_size, 0)
          return dim_size

      # caculate output shape
      batch_size = inputs.get_shape()[0].value
      height = inputs.get_shape()[1].value
      width = inputs.get_shape()[2].value
      out_height = get_deconv_dim(height, stride_h, kernel_h, padding)
      out_width = get_deconv_dim(width, stride_w, kernel_w, padding)
      output_shape = [batch_size, out_height, out_width, num_output_channels]

      outputs = tf.nn.conv2d_transpose(inputs, kernel, output_shape,
                            [1, stride_h, stride_w, 1],
                            padding=padding)
      biases = _variable_on_cpu('biases', [num_output_channels],
                                tf.constant_initializer(0.0))
      outputs = tf.nn.bias_add(outputs, biases)

      if bn:
        outputs = batch_norm_for_conv2d(outputs, is_training,
                                        bn_decay=bn_decay, scope='bn')

      if activation_fn is not None:
        outputs = activation_fn(outputs)
      return outputs

  
def conv3d(inputs,
          num_output_channels,
          kernel_size,
          scope,
          stride=[1, 1, 1],
          padding='SAME',
          use_xavier=True,
          stddev=1e-3,
          weight_decay=0.0,
          activation_fn=tf.nn.relu,
          bn=False,
          bn_decay=None,
          is_training=None):
  """ 3D convolution with non-linear operation.

  Args:
    inputs: 5-D tensor variable BxDxHxWxC
    num_output_channels: int
    kernel_size: a list of 3 ints
    scope: string
    stride: a list of 3 ints
    padding: 'SAME' or 'VALID'
    use_xavier: bool, use xavier_initializer if true
    stddev: float, stddev for truncated_normal init
    weight_decay: float
    activation_fn: function
    bn: bool, whether to use batch norm
    bn_decay: float or float tensor variable in [0,1]
    is_training: bool Tensor variable

  Returns:
    Variable tensor
  """
  with tf.variable_scope(scope) as sc:
    kernel_d, kernel_h, kernel_w = kernel_size
    num_in_channels = inputs.get_shape()[-1].value
    kernel_shape = [kernel_d, kernel_h, kernel_w,
                    num_in_channels, num_output_channels]
    kernel = _variable_with_weight_decay('weights',
                                        shape=kernel_shape,
                                        use_xavier=use_xavier,
                                        stddev=stddev,
                                        wd=weight_decay)
    stride_d, stride_h, stride_w = stride
    outputs = tf.nn.conv3d(inputs, kernel,
                          [1, stride_d, stride_h, stride_w, 1],
                          padding=padding)
    biases = _variable_on_cpu('biases', [num_output_channels],
                              tf.constant_initializer(0.0))
    outputs = tf.nn.bias_add(outputs, biases)
    
    if bn:
      outputs = batch_norm_for_conv3d(outputs, is_training,
                                      bn_decay=bn_decay, scope='bn')

    if activation_fn is not None:
      outputs = activation_fn(outputs)
    return outputs


def fully_connected(inputs,
                    num_outputs,
                    scope,
                    use_xavier=True,
                    stddev=1e-3,
                    weight_decay=0.0,
                    activation_fn=tf.nn.relu,
                    bn=False,
                    bn_decay=None,
                    is_training=None):
  """ Fully connected layer with non-linear operation.
  
  Args:
    inputs: 2-D tensor BxN
    num_outputs: int
  
  Returns:
    Variable tensor of size B x num_outputs.
  """
  with tf.variable_scope(scope) as sc:
    num_input_units = inputs.get_shape()[-1].value
    weights = _variable_with_weight_decay('weights',
                                          shape=[num_input_units, num_outputs],
                                          use_xavier=use_xavier,
                                          stddev=stddev,
                                          wd=weight_decay)
    outputs = tf.matmul(inputs, weights)
    biases = _variable_on_cpu('biases', [num_outputs],
                            tf.constant_initializer(0.0))
    outputs = tf.nn.bias_add(outputs, biases)
    
    if bn:
      outputs = batch_norm_for_fc(outputs, is_training, bn_decay, 'bn')

    if activation_fn is not None:
      outputs = activation_fn(outputs)
    return outputs


def max_pool2d(inputs,
              kernel_size,
              scope,
              stride=[2, 2],
              padding='VALID'):
  """ 2D max pooling.

  Args:
    inputs: 4-D tensor BxHxWxC
    kernel_size: a list of 2 ints
    stride: a list of 2 ints
  
  Returns:
    Variable tensor
  """
  with tf.variable_scope(scope) as sc:
    kernel_h, kernel_w = kernel_size
    stride_h, stride_w = stride
    outputs = tf.nn.max_pool(inputs,
                            ksize=[1, kernel_h, kernel_w, 1],
                            strides=[1, stride_h, stride_w, 1],
                            padding=padding,
                            name=sc.name)
    return outputs


def avg_pool2d(inputs,
              kernel_size,
              scope,
              stride=[2, 2],
              padding='VALID'):
  """ 2D avg pooling.

  Args:
    inputs: 4-D tensor BxHxWxC
    kernel_size: a list of 2 ints
    stride: a list of 2 ints
  
  Returns:
    Variable tensor
  """
  with tf.variable_scope(scope) as sc:
    kernel_h, kernel_w = kernel_size
    stride_h, stride_w = stride
    outputs = tf.nn.avg_pool(inputs,
                            ksize=[1, kernel_h, kernel_w, 1],
                            strides=[1, stride_h, stride_w, 1],
                            padding=padding,
                            name=sc.name)
    return outputs


def max_pool3d(inputs,
              kernel_size,
              scope,
              stride=[2, 2, 2],
              padding='VALID'):
  """ 3D max pooling.

  Args:
    inputs: 5-D tensor BxDxHxWxC
    kernel_size: a list of 3 ints
    stride: a list of 3 ints
  
  Returns:
    Variable tensor
  """
  with tf.variable_scope(scope) as sc:
    kernel_d, kernel_h, kernel_w = kernel_size
    stride_d, stride_h, stride_w = stride
    outputs = tf.nn.max_pool3d(inputs,
                              ksize=[1, kernel_d, kernel_h, kernel_w, 1],
                              strides=[1, stride_d, stride_h, stride_w, 1],
                              padding=padding,
                              name=sc.name)
    return outputs


def avg_pool3d(inputs,
              kernel_size,
              scope,
              stride=[2, 2, 2],
              padding='VALID'):
  """ 3D avg pooling.

  Args:
    inputs: 5-D tensor BxDxHxWxC
    kernel_size: a list of 3 ints
    stride: a list of 3 ints
  
  Returns:
    Variable tensor
  """
  with tf.variable_scope(scope) as sc:
    kernel_d, kernel_h, kernel_w = kernel_size
    stride_d, stride_h, stride_w = stride
    outputs = tf.nn.avg_pool3d(inputs,
                              ksize=[1, kernel_d, kernel_h, kernel_w, 1],
                              strides=[1, stride_d, stride_h, stride_w, 1],
                              padding=padding,
                              name=sc.name)
    return outputs


def batch_norm_template(inputs, is_training, scope, moments_dims, bn_decay):
  """ Batch normalization on convolutional maps and beyond...
  Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
  
  Args:
      inputs:        Tensor, k-D input ... x C could be BC or BHWC or BDHWC
      is_training:   boolean tf.Varialbe, true indicates training phase
      scope:         string, variable scope
      moments_dims:  a list of ints, indicating dimensions for moments calculation
      bn_decay:      float or float tensor variable, controling moving average weight
  Return:
      normed:        batch-normalized maps
  """
  with tf.variable_scope(scope) as sc:
    num_channels = inputs.get_shape()[-1].value
    beta = tf.Variable(tf.constant(0.0, shape=[num_channels]),
                      name='beta', trainable=True)
    gamma = tf.Variable(tf.constant(1.0, shape=[num_channels]),
                        name='gamma', trainable=True)
    batch_mean, batch_var = tf.nn.moments(inputs, moments_dims, name='moments')
    decay = bn_decay if bn_decay is not None else 0.9
    ema = tf.train.ExponentialMovingAverage(decay=decay)
    # Operator that maintains moving averages of variables.
    ema_apply_op = tf.cond(is_training,
                          lambda: ema.apply([batch_mean, batch_var]),
                          lambda: tf.no_op())
    
    # Update moving average and return current batch's avg and var.
    def mean_var_with_update():
      with tf.control_dependencies([ema_apply_op]):
        return tf.identity(batch_mean), tf.identity(batch_var)
    
    # ema.average returns the Variable holding the average of var.
    mean, var = tf.cond(is_training,
                        mean_var_with_update,
                        lambda: (ema.average(batch_mean), ema.average(batch_var)))
    normed = tf.nn.batch_normalization(inputs, mean, var, beta, gamma, 1e-3)
  return normed


def batch_norm_for_fc(inputs, is_training, bn_decay, scope):
  """ Batch normalization on FC data.
  
  Args:
      inputs:      Tensor, 2D BxC input
      is_training: boolean tf.Varialbe, true indicates training phase
      bn_decay:    float or float tensor variable, controling moving average weight
      scope:       string, variable scope
  Return:
      normed:      batch-normalized maps
  """
  return batch_norm_template(inputs, is_training, scope, [0,], bn_decay)


def batch_norm_for_conv1d(inputs, is_training, bn_decay, scope):
  """ Batch normalization on 1D convolutional maps.
  
  Args:
      inputs:      Tensor, 3D BLC input maps
      is_training: boolean tf.Varialbe, true indicates training phase
      bn_decay:    float or float tensor variable, controling moving average weight
      scope:       string, variable scope
  Return:
      normed:      batch-normalized maps
  """
  return batch_norm_template(inputs, is_training, scope, [0,1], bn_decay)


def batch_norm_for_conv2d(inputs, is_training, bn_decay, scope):
  """ Batch normalization on 2D convolutional maps.
  
  Args:
      inputs:      Tensor, 4D BHWC input maps
      is_training: boolean tf.Varialbe, true indicates training phase
      bn_decay:    float or float tensor variable, controling moving average weight
      scope:       string, variable scope
  Return:
      normed:      batch-normalized maps
  """
  return batch_norm_template(inputs, is_training, scope, [0,1,2], bn_decay)


def batch_norm_for_conv3d(inputs, is_training, bn_decay, scope):
  """ Batch normalization on 3D convolutional maps.
  
  Args:
      inputs:      Tensor, 5D BDHWC input maps
      is_training: boolean tf.Varialbe, true indicates training phase
      bn_decay:    float or float tensor variable, controling moving average weight
      scope:       string, variable scope
  Return:
      normed:      batch-normalized maps
  """
  return batch_norm_template(inputs, is_training, scope, [0,1,2,3], bn_decay)


def dropout(inputs,
            is_training,
            scope,
            keep_prob=0.5,
            noise_shape=None):
  """ Dropout layer.

  Args:
    inputs: tensor
    is_training: boolean tf.Variable
    scope: string
    keep_prob: float in [0,1]
    noise_shape: list of ints

  Returns:
    tensor variable
  """
  with tf.variable_scope(scope) as sc:
    outputs = tf.cond(is_training,
                      lambda: tf.nn.dropout(inputs, keep_prob, noise_shape),
                      lambda: inputs)
    return outputs


class TF_util():
  @staticmethod
  def _variable_on_cpu(name, shape, initializer, use_fp16=False):
    """Helper to create a Variable stored on CPU memory.
    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable
    Returns:
      Variable Tensor
    """
    with tf.device('/cpu:0'):
      dtype = tf.float16 if use_fp16 else tf.float32
      var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
    return var

  @staticmethod
  def _variable_with_weight_decay(name, shape, stddev, wd, use_xavier=True):
    """Helper to create an initialized Variable with weight decay.

    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.

    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.
      use_xavier: bool, whether to use xavier initializer

    Returns:
      Variable Tensor
    """
    if use_xavier:
      initializer = tf.contrib.layers.xavier_initializer()
    else:
      initializer = tf.truncated_normal_initializer(stddev=stddev)
    var = _variable_on_cpu(name, shape, initializer)
    if wd is not None:
      weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
      tf.add_to_collection('losses', weight_decay)
    return var


  @staticmethod
  def conv1d(inputs,
            num_output_channels,
            kernel_size,
            scope,
            stride=1,
            padding='SAME',
            use_xavier=True,
            stddev=1e-3,
            weight_decay=0.0,
            activation_fn=tf.nn.relu,
            bn=False,
            bn_decay=None,
            is_training=None):
    """ 1D convolution with non-linear operation.

    Args:
      inputs: 3-D tensor variable BxLxC
      num_output_channels: int
      kernel_size: int
      scope: string
      stride: int
      padding: 'SAME' or 'VALID'
      use_xavier: bool, use xavier_initializer if true
      stddev: float, stddev for truncated_normal init
      weight_decay: float
      activation_fn: function
      bn: bool, whether to use batch norm
      bn_decay: float or float tensor variable in [0,1]
      is_training: bool Tensor variable

    Returns:
      Variable tensor
    """
    with tf.variable_scope(scope) as sc:
      num_in_channels = inputs.get_shape()[-1].value
      kernel_shape = [kernel_size,
                      num_in_channels, num_output_channels]
      kernel = _variable_with_weight_decay('weights',
                                          shape=kernel_shape,
                                          use_xavier=use_xavier,
                                          stddev=stddev,
                                          wd=weight_decay)
      outputs = tf.nn.conv1d(inputs, kernel,
                            stride=stride,
                            padding=padding)
      biases = _variable_on_cpu('biases', [num_output_channels],
                                tf.constant_initializer(0.0))
      outputs = tf.nn.bias_add(outputs, biases)

      if bn:
        outputs = batch_norm_for_conv1d(outputs, is_training,
                                        bn_decay=bn_decay, scope='bn')

      if activation_fn is not None:
        outputs = activation_fn(outputs)
      return outputs


  @staticmethod
  def conv2d(inputs,
            num_output_channels,
            kernel_size,
            scope,
            stride=[1, 1],
            padding='SAME',
            use_xavier=True,
            stddev=1e-3,
            weight_decay=0.0,
            activation_fn=tf.nn.relu,
            bn=False,
            bn_decay=None,
            is_training=None):
    """ 2D convolution with non-linear operation.

    Args:
      inputs: 4-D tensor variable BxHxWxC
      num_output_channels: int
      kernel_size: a list of 2 ints
      scope: string
      stride: a list of 2 ints
      padding: 'SAME' or 'VALID'
      use_xavier: bool, use xavier_initializer if true
      stddev: float, stddev for truncated_normal init
      weight_decay: float
      activation_fn: function
      bn: bool, whether to use batch norm
      bn_decay: float or float tensor variable in [0,1]
      is_training: bool Tensor variable

    Returns:
      Variable tensor
    """
    
    with tf.variable_scope(scope) as sc:
        kernel_h, kernel_w = kernel_size
        num_in_channels = inputs.get_shape()[-1].value
        kernel_shape = [kernel_h, kernel_w,
                        num_in_channels, num_output_channels]
        kernel = _variable_with_weight_decay('weights',
                                            shape=kernel_shape,
                                            use_xavier=use_xavier,
                                            stddev=stddev,
                                            wd=weight_decay)
        stride_h, stride_w = stride
        outputs = tf.nn.conv2d(inputs, kernel,
                              [1, stride_h, stride_w, 1],
                              padding=padding)
        biases = _variable_on_cpu('biases', [num_output_channels],
                                  tf.constant_initializer(0.0))
        outputs = tf.nn.bias_add(outputs, biases)

        if bn:
          outputs = batch_norm_for_conv2d(outputs, is_training,
                                          bn_decay=bn_decay, scope='bn')

        if activation_fn is not None:
          outputs = activation_fn(outputs)
        return outputs


  @staticmethod
  def conv2d_transpose(inputs,
                      num_output_channels,
                      kernel_size,
                      scope,
                      stride=[1, 1],
                      padding='SAME',
                      use_xavier=True,
                      stddev=1e-3,
                      weight_decay=0.0,
                      activation_fn=tf.nn.relu,
                      bn=False,
                      bn_decay=None,
                      is_training=None):
    """ 2D convolution transpose with non-linear operation.

    Args:
      inputs: 4-D tensor variable BxHxWxC
      num_output_channels: int
      kernel_size: a list of 2 ints
      scope: string
      stride: a list of 2 ints
      padding: 'SAME' or 'VALID'
      use_xavier: bool, use xavier_initializer if true
      stddev: float, stddev for truncated_normal init
      weight_decay: float
      activation_fn: function
      bn: bool, whether to use batch norm
      bn_decay: float or float tensor variable in [0,1]
      is_training: bool Tensor variable

    Returns:
      Variable tensor

    Note: conv2d(conv2d_transpose(a, num_out, ksize, stride), a.shape[-1], ksize, stride) == a
    """
    with tf.variable_scope(scope) as sc:
        kernel_h, kernel_w = kernel_size
        num_in_channels = inputs.get_shape()[-1].value
        kernel_shape = [kernel_h, kernel_w,
                        num_output_channels, num_in_channels] # reversed to conv2d
        kernel = _variable_with_weight_decay('weights',
                                            shape=kernel_shape,
                                            use_xavier=use_xavier,
                                            stddev=stddev,
                                            wd=weight_decay)
        stride_h, stride_w = stride
        
        # from slim.convolution2d_transpose
        def get_deconv_dim(dim_size, stride_size, kernel_size, padding):
            dim_size *= stride_size

            if padding == 'VALID' and dim_size is not None:
              dim_size += max(kernel_size - stride_size, 0)
            return dim_size

        # caculate output shape
        batch_size = inputs.get_shape()[0].value
        height = inputs.get_shape()[1].value
        width = inputs.get_shape()[2].value
        out_height = get_deconv_dim(height, stride_h, kernel_h, padding)
        out_width = get_deconv_dim(width, stride_w, kernel_w, padding)
        output_shape = [batch_size, out_height, out_width, num_output_channels]

        outputs = tf.nn.conv2d_transpose(inputs, kernel, output_shape,
                              [1, stride_h, stride_w, 1],
                              padding=padding)
        biases = _variable_on_cpu('biases', [num_output_channels],
                                  tf.constant_initializer(0.0))
        outputs = tf.nn.bias_add(outputs, biases)

        if bn:
          outputs = batch_norm_for_conv2d(outputs, is_training,
                                          bn_decay=bn_decay, scope='bn')

        if activation_fn is not None:
          outputs = activation_fn(outputs)
        return outputs

    
  @staticmethod
  def conv3d(inputs,
            num_output_channels,
            kernel_size,
            scope,
            stride=[1, 1, 1],
            padding='SAME',
            use_xavier=True,
            stddev=1e-3,
            weight_decay=0.0,
            activation_fn=tf.nn.relu,
            bn=False,
            bn_decay=None,
            is_training=None):
    """ 3D convolution with non-linear operation.

    Args:
      inputs: 5-D tensor variable BxDxHxWxC
      num_output_channels: int
      kernel_size: a list of 3 ints
      scope: string
      stride: a list of 3 ints
      padding: 'SAME' or 'VALID'
      use_xavier: bool, use xavier_initializer if true
      stddev: float, stddev for truncated_normal init
      weight_decay: float
      activation_fn: function
      bn: bool, whether to use batch norm
      bn_decay: float or float tensor variable in [0,1]
      is_training: bool Tensor variable

    Returns:
      Variable tensor
    """
    with tf.variable_scope(scope) as sc:
      kernel_d, kernel_h, kernel_w = kernel_size
      num_in_channels = inputs.get_shape()[-1].value
      kernel_shape = [kernel_d, kernel_h, kernel_w,
                      num_in_channels, num_output_channels]
      kernel = _variable_with_weight_decay('weights',
                                          shape=kernel_shape,
                                          use_xavier=use_xavier,
                                          stddev=stddev,
                                          wd=weight_decay)
      stride_d, stride_h, stride_w = stride
      outputs = tf.nn.conv3d(inputs, kernel,
                            [1, stride_d, stride_h, stride_w, 1],
                            padding=padding)
      biases = _variable_on_cpu('biases', [num_output_channels],
                                tf.constant_initializer(0.0))
      outputs = tf.nn.bias_add(outputs, biases)
      
      if bn:
        outputs = batch_norm_for_conv3d(outputs, is_training,
                                        bn_decay=bn_decay, scope='bn')

      if activation_fn is not None:
        outputs = activation_fn(outputs)
      return outputs


  @staticmethod
  def fully_connected(inputs,
                      num_outputs,
                      scope,
                      use_xavier=True,
                      stddev=1e-3,
                      weight_decay=0.0,
                      activation_fn=tf.nn.relu,
                      bn=False,
                      bn_decay=None,
                      is_training=None):
    """ Fully connected layer with non-linear operation.
    
    Args:
      inputs: 2-D tensor BxN
      num_outputs: int
    
    Returns:
      Variable tensor of size B x num_outputs.
    """
    with tf.variable_scope(scope) as sc:
      num_input_units = inputs.get_shape()[-1].value
      weights = _variable_with_weight_decay('weights',
                                            shape=[num_input_units, num_outputs],
                                            use_xavier=use_xavier,
                                            stddev=stddev,
                                            wd=weight_decay)
      outputs = tf.matmul(inputs, weights)
      biases = _variable_on_cpu('biases', [num_outputs],
                              tf.constant_initializer(0.0))
      outputs = tf.nn.bias_add(outputs, biases)
      
      if bn:
        outputs = batch_norm_for_fc(outputs, is_training, bn_decay, 'bn')

      if activation_fn is not None:
        outputs = activation_fn(outputs)
      return outputs

  @staticmethod
  def max_pool2d(inputs,
                kernel_size,
                scope,
                stride=[2, 2],
                padding='VALID'):
    """ 2D max pooling.

    Args:
      inputs: 4-D tensor BxHxWxC
      kernel_size: a list of 2 ints
      stride: a list of 2 ints
    
    Returns:
      Variable tensor
    """
    with tf.variable_scope(scope) as sc:
      kernel_h, kernel_w = kernel_size
      stride_h, stride_w = stride
      outputs = tf.nn.max_pool(inputs,
                              ksize=[1, kernel_h, kernel_w, 1],
                              strides=[1, stride_h, stride_w, 1],
                              padding=padding,
                              name=sc.name)
      return outputs


  @staticmethod
  def avg_pool2d(inputs,
                kernel_size,
                scope,
                stride=[2, 2],
                padding='VALID'):
    """ 2D avg pooling.

    Args:
      inputs: 4-D tensor BxHxWxC
      kernel_size: a list of 2 ints
      stride: a list of 2 ints
    
    Returns:
      Variable tensor
    """
    with tf.variable_scope(scope) as sc:
      kernel_h, kernel_w = kernel_size
      stride_h, stride_w = stride
      outputs = tf.nn.avg_pool(inputs,
                              ksize=[1, kernel_h, kernel_w, 1],
                              strides=[1, stride_h, stride_w, 1],
                              padding=padding,
                              name=sc.name)
      return outputs

  @staticmethod
  def max_pool3d(inputs,
                kernel_size,
                scope,
                stride=[2, 2, 2],
                padding='VALID'):
    """ 3D max pooling.

    Args:
      inputs: 5-D tensor BxDxHxWxC
      kernel_size: a list of 3 ints
      stride: a list of 3 ints
    
    Returns:
      Variable tensor
    """
    with tf.variable_scope(scope) as sc:
      kernel_d, kernel_h, kernel_w = kernel_size
      stride_d, stride_h, stride_w = stride
      outputs = tf.nn.max_pool3d(inputs,
                                ksize=[1, kernel_d, kernel_h, kernel_w, 1],
                                strides=[1, stride_d, stride_h, stride_w, 1],
                                padding=padding,
                                name=sc.name)
      return outputs


  @staticmethod
  def avg_pool3d(inputs,
                kernel_size,
                scope,
                stride=[2, 2, 2],
                padding='VALID'):
    """ 3D avg pooling.

    Args:
      inputs: 5-D tensor BxDxHxWxC
      kernel_size: a list of 3 ints
      stride: a list of 3 ints
    
    Returns:
      Variable tensor
    """
    with tf.variable_scope(scope) as sc:
      kernel_d, kernel_h, kernel_w = kernel_size
      stride_d, stride_h, stride_w = stride
      outputs = tf.nn.avg_pool3d(inputs,
                                ksize=[1, kernel_d, kernel_h, kernel_w, 1],
                                strides=[1, stride_d, stride_h, stride_w, 1],
                                padding=padding,
                                name=sc.name)
      return outputs




  @staticmethod
  def batch_norm_template(inputs, is_training, scope, moments_dims, bn_decay):
    """ Batch normalization on convolutional maps and beyond...
    Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
    
    Args:
        inputs:        Tensor, k-D input ... x C could be BC or BHWC or BDHWC
        is_training:   boolean tf.Varialbe, true indicates training phase
        scope:         string, variable scope
        moments_dims:  a list of ints, indicating dimensions for moments calculation
        bn_decay:      float or float tensor variable, controling moving average weight
    Return:
        normed:        batch-normalized maps
    """
    with tf.variable_scope(scope) as sc:
      num_channels = inputs.get_shape()[-1].value
      beta = tf.Variable(tf.constant(0.0, shape=[num_channels]),
                        name='beta', trainable=True)
      gamma = tf.Variable(tf.constant(1.0, shape=[num_channels]),
                          name='gamma', trainable=True)
      batch_mean, batch_var = tf.nn.moments(inputs, moments_dims, name='moments')
      decay = bn_decay if bn_decay is not None else 0.9
      ema = tf.train.ExponentialMovingAverage(decay=decay)
      # Operator that maintains moving averages of variables.
      ema_apply_op = tf.cond(is_training,
                            lambda: ema.apply([batch_mean, batch_var]),
                            lambda: tf.no_op())
      
      # Update moving average and return current batch's avg and var.
      def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
          return tf.identity(batch_mean), tf.identity(batch_var)
      
      # ema.average returns the Variable holding the average of var.
      mean, var = tf.cond(is_training,
                          mean_var_with_update,
                          lambda: (ema.average(batch_mean), ema.average(batch_var)))
      normed = tf.nn.batch_normalization(inputs, mean, var, beta, gamma, 1e-3)
    return normed


  @staticmethod
  def batch_norm_for_fc(inputs, is_training, bn_decay, scope):
    """ Batch normalization on FC data.
    
    Args:
        inputs:      Tensor, 2D BxC input
        is_training: boolean tf.Varialbe, true indicates training phase
        bn_decay:    float or float tensor variable, controling moving average weight
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    return batch_norm_template(inputs, is_training, scope, [0,], bn_decay)


  @staticmethod
  def batch_norm_for_conv1d(inputs, is_training, bn_decay, scope):
    """ Batch normalization on 1D convolutional maps.
    
    Args:
        inputs:      Tensor, 3D BLC input maps
        is_training: boolean tf.Varialbe, true indicates training phase
        bn_decay:    float or float tensor variable, controling moving average weight
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    return batch_norm_template(inputs, is_training, scope, [0,1], bn_decay)



  @staticmethod
  def batch_norm_for_conv2d(inputs, is_training, bn_decay, scope):
    """ Batch normalization on 2D convolutional maps.
    
    Args:
        inputs:      Tensor, 4D BHWC input maps
        is_training: boolean tf.Varialbe, true indicates training phase
        bn_decay:    float or float tensor variable, controling moving average weight
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    return batch_norm_template(inputs, is_training, scope, [0,1,2], bn_decay)


  @staticmethod
  def batch_norm_for_conv3d(inputs, is_training, bn_decay, scope):
    """ Batch normalization on 3D convolutional maps.
    
    Args:
        inputs:      Tensor, 5D BDHWC input maps
        is_training: boolean tf.Varialbe, true indicates training phase
        bn_decay:    float or float tensor variable, controling moving average weight
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    return batch_norm_template(inputs, is_training, scope, [0,1,2,3], bn_decay)


  @staticmethod
  def dropout(inputs,
              is_training,
              scope,
              keep_prob=0.5,
              noise_shape=None):
    """ Dropout layer.

    Args:
      inputs: tensor
      is_training: boolean tf.Variable
      scope: string
      keep_prob: float in [0,1]
      noise_shape: list of ints

    Returns:
      tensor variable
    """
    with tf.variable_scope(scope) as sc:
      outputs = tf.cond(is_training,
                        lambda: tf.nn.dropout(inputs, keep_prob, noise_shape),
                        lambda: inputs)
      return outputs

tf_util = TF_util()

# Data loading

In [4]:
# Global constants:

BATCH_SIZE = 32
NUM_POINT = 1024
MAX_EPOCH = 250
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 0
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 20000
DECAY_RATE = 0.8

MAX_NUM_POINT = 2048
NUN_CLASSES = 40

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

In [16]:
def even_sample(mesh, n, r):
    samples = np.zeros((1, 1))
    while samples.shape[0] != n:
        samples = sample_surface_even(mesh, n, r)[0]
        r /= 10
    return samples

In [ ]:
dir_path = '/content/drive/My Drive/Technion/Master/Geometric Learning/Final Project/ModelNet10/ModelNet10/'
classes = os.listdir(dir_path)
train_labels, test_labels = [], []
train_data, test_data = [], []
train_faces, test_faces = [], []

for c in tqdm(classes):

  # Train data loading:
  for mesh in tqdm(os.listdir(dir_path + c + '/train')):
    train_labels += [c]
    train_mesh = trimesh.load_mesh(dir_path + c + '/train/' + mesh)
    train_data += [np.array(even_sample(train_mesh, NUM_POINT, 1))]

  # Test data loading:
  for mesh in tqdm(os.listdir(dir_path + c + '/test')):
    test_labels += [c]
    test_mesh = trimesh.load_mesh(dir_path + c + '/test/' + mesh)
    test_data += [np.array(even_sample(test_mesh, NUM_POINT, 1))]

In [38]:
train_data_concatenated = np.concatenate([a[np.newaxis, :, :] for a in train_data], axis=0)
test_data_concatenated = np.concatenate([a[np.newaxis, :, :] for a in test_data], axis=0)

In [54]:
with open('/content/drive/My Drive/Technion/Master/Geometric Learning/Final Project/train_data.npy', 'wb') as train_file:
  np.save(train_file, train_data_concatenated)

with open('/content/drive/My Drive/Technion/Master/Geometric Learning/Final Project/test_data.npy', 'wb') as test_file:
  np.save(test_file, test_data_concatenated)

# Run basic experiments

## Classic **PointNet**

## Classic **Momenet**


In [ ]:
class Momenet():
  def placeholder_inputs(batch_size, num_point):
      pointclouds_pl = tf.placeholder(tf.float32, shape=(batch_size, num_point, 9))
      labels_pl = tf.placeholder(tf.int32, shape=(batch_size))
      return pointclouds_pl, labels_pl


  def get_model(point_cloud, is_training, bn_decay=None):
      """ Classification PointNet, input is BxNx3, output Bx40 """
      batch_size = point_cloud.get_shape()[0].value
      num_point = point_cloud.get_shape()[1].value
      end_points = {}
      input_image = tf.expand_dims(point_cloud, -1)
      net = 1
      # Point functions (MLP implemented as conv2d)
      net = tf_util.conv2d(input_image, 64, [1,9], 'conv1',
                          padding='VALID', stride=[1,1],
                          bn=True, is_training=is_training,
                          bn_decay=bn_decay)
      net = tf_util.conv2d(net, 64, [1,1],
                          padding='VALID', stride=[1,1],
                          bn=True, is_training=is_training,
                          scope='conv2', bn_decay=bn_decay)
      net = tf_util.conv2d(net, 64, [1,1],
                          padding='VALID', stride=[1,1],
                          bn=True, is_training=is_training,
                          scope='conv3', bn_decay=bn_decay)
      net = tf_util.conv2d(net, 128, [1,1],
                          padding='VALID', stride=[1,1],
                          bn=True, is_training=is_training,
                          scope='conv4', bn_decay=bn_decay)
      net = tf_util.conv2d(net, 1024, [1,1],
                          padding='VALID', stride=[1,1],
                          bn=True, is_training=is_training,
                          scope='conv5', bn_decay=bn_decay)

      # Symmetric function: max pooling
      net = tf_util.max_pool2d(net, [num_point,1],
                              padding='VALID', scope='maxpool')
      
      # MLP on global point cloud vector
      net = tf.reshape(net, [batch_size, -1])
      net = tf_util.fully_connected(net, 512, bn=True, is_training=is_training,
                                    scope='fc1', bn_decay=bn_decay)
      net = tf_util.fully_connected(net, 256, bn=True, is_training=is_training,
                                    scope='fc2', bn_decay=bn_decay)
      net = tf_util.dropout(net, keep_prob=0.7, is_training=is_training,
                            scope='dp1')
      net = tf_util.fully_connected(net, 40, activation_fn=None, scope='fc3')

      return net, end_points


  def get_loss(pred, label, end_points):
      """ pred: B*NUM_CLASSES,
          label: B, """
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=label)
      classify_loss = tf.reduce_mean(loss)
      tf.summary.scalar('classify loss', classify_loss)
      return classify_loss

MODEL = Momenet

In [ ]:
LOG_DIR = '/content/drive/My Drive/Technion/Master/Geometric Learning/Final Project/Momenet/log'
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')

In [ ]:
# Sanity check that get_model works
with tf.Graph().as_default():
  inputs = tf.zeros((32, 1024, 9))
  outputs = MODEL.get_model(inputs, tf.constant(True))
  print(outputs)

(<tf.Tensor 'fc3/BiasAdd:0' shape=(32, 40) dtype=float32>, {})


In [ ]:
def add_moments(data, order=2):
  if order == 1:
    return data
  elif order == 2:
    data_moments = np.zeros((np.shape(data)[0], np.shape(data)[1], 9))
  elif order == 3:
    raise ValueError('3rd order moments are not prepared yet!')
  
  data_moments[:, :, 0:3] = data
  data_moments[:, :, 3] = data_moments[:, :, 0] * data_moments[:, :, 0]
  data_moments[:, :, 4] = data_moments[:, :, 1] * data_moments[:, :, 1]
  data_moments[:, :, 5] = data_moments[:, :, 2] * data_moments[:, :, 2]
  data_moments[:, :, 6] = data_moments[:, :, 0] * data_moments[:, :, 1]
  data_moments[:, :, 7] = data_moments[:, :, 0] * data_moments[:, :, 2]
  data_moments[:, :, 8] = data_moments[:, :, 1] * data_moments[:, :, 2]

  return data_moments

In [ ]:
def get_bn_decay(batch):
  bn_momentum = tf.train.exponential_decay(
                    BN_INIT_DECAY,
                    batch*BATCH_SIZE,
                    BN_DECAY_DECAY_STEP,
                    BN_DECAY_DECAY_RATE,
                    staircase=True)
  bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
  return bn_decay

In [ ]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)

In [ ]:
def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!
    return learning_rate   

In [ ]:
def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    # Shuffle train files
    train_file_idxs = np.arange(0, len(TRAIN_FILES))
    np.random.shuffle(train_file_idxs)
    
    # for fn in range(len(TRAIN_FILES)):
    #     log_string('----' + str(fn) + '-----')
    #     current_data, current_label = provider.loadDataFile(TRAIN_FILES[train_file_idxs[fn]])
    #     current_data = current_data[:,0:NUM_POINT,:]
    #     current_data, current_label, _ = provider.shuffle_data(current_data, np.squeeze(current_label))            
    #     current_label = np.squeeze(current_label)
        
        
        
    #     file_size = current_data.shape[0]
    #     num_batches = file_size // BATCH_SIZE
        
    #     total_correct = 0
    #     total_seen = 0
    #     loss_sum = 0
       
    #     for batch_idx in range(num_batches):
    #         start_idx = batch_idx * BATCH_SIZE
    #         end_idx = (batch_idx+1) * BATCH_SIZE

    #         # Augment batched point clouds by rotation and jittering
    #         rotated_data = provider.rotate_point_cloud(current_data[start_idx:end_idx, :, :])
    #         jittered_data = provider.jitter_point_cloud(rotated_data)
    #         feed_dict = {ops['pointclouds_pl']: jittered_data,
    #                      ops['labels_pl']: current_label[start_idx:end_idx],
    #                      ops['is_training_pl']: is_training,}
            
           
            
    #         summary, step, _, loss_val, pred_val = sess.run([ops['merged'], ops['step'],
    #             ops['train_op'], ops['loss'], ops['pred']], feed_dict=feed_dict)
            
            
    #         train_writer.add_summary(summary, step)
    #         pred_val = np.argmax(pred_val, 1)
    #         correct = np.sum(pred_val == current_label[start_idx:end_idx])
    #         total_correct += correct
    #         total_seen += BATCH_SIZE
    #         loss_sum += loss_val
        
    #     log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    #     log_string('accuracy: %f' % (total_correct / float(total_seen)))

In [ ]:
def train():
  with tf.Graph().as_default():
    with tf.device('/gpu:' + str(GPU_INDEX)):
      pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
      is_training_pl = tf.placeholder(tf.bool, shape=())
      print(is_training_pl)
      
      # Note the global_step=batch parameter to minimize. 
      # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
      batch = tf.Variable(0)
      bn_decay = get_bn_decay(batch)
      tf.summary.scalar('bn_decay', bn_decay)

      # Get model and loss 
      pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
      loss = MODEL.get_loss(pred, labels_pl, end_points)
      tf.summary.scalar('loss', loss)

      correct = tf.equal(tf.argmax(pred, 1), tf.to_int64(labels_pl))
      accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE)
      tf.summary.scalar('accuracy', accuracy)

      # Get training operator
      learning_rate = get_learning_rate(batch)
      tf.summary.scalar('learning_rate', learning_rate)
      if OPTIMIZER == 'momentum':
          optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
      elif OPTIMIZER == 'adam':
          optimizer = tf.train.AdamOptimizer(learning_rate)
      train_op = optimizer.minimize(loss, global_step=batch)
      
      # Add ops to save and restore all the variables.
      saver = tf.train.Saver()
          
      # Create a session
      config = tf.ConfigProto()
      config.gpu_options.allow_growth = True
      config.allow_soft_placement = True
      config.log_device_placement = False
      sess = tf.Session(config=config)

      # Add summary writers
      #merged = tf.merge_all_summaries()
      merged = tf.summary.merge_all()
      train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                                sess.graph)
      test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

      # Init variables
      init = tf.global_variables_initializer()
      # To fix the bug introduced in TF 0.12.1 as in
      # http://stackoverflow.com/questions/41543774/invalidargumenterror-for-tensor-bool-tensorflow-0-12-1
      #sess.run(init)
      sess.run(init, {is_training_pl: True})

      ops = {'pointclouds_pl': pointclouds_pl,
              'labels_pl': labels_pl,
              'is_training_pl': is_training_pl,
              'pred': pred,
              'loss': loss,
              'train_op': train_op,
              'merged': merged,
              'step': batch}
      best_acc=0
      for epoch in range(MAX_EPOCH):
          log_string('**** EPOCH %03d ****' % (epoch))
          # sys.stdout.flush()
            
          train_one_epoch(sess, ops, train_writer)
          # acc = eval_one_epoch(sess, ops, test_writer)
          
          # if acc > best_acc:
          #     save_path = saver.save(sess, os.path.join(LOG_DIR, "best_model.ckpt"))
          #     log_string("Best Model saved in file: %s" % save_path)
          #     best_acc = acc
          
          # # Save the variables to disk.
          # if epoch % 10 == 0:
          #     save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
          #     log_string("Model saved in file: %s" % save_path)

train()

## $1^{st}$, $2^{nd}$ and $3^{rd}$ order moments

In [ ]:
def add_moments(data, order=2):
  if order == 1:
    return data
  elif order == 2:
    data_moments = np.zeros((np.shape(data)[0], np.shape(data)[1], 9))
  elif order == 3:
    raise ValueError('3rd order moments are not prepared yet!')
  
  data_moments[:, :, 0:3] = data
  data_moments[:, :, 3] = data_moments[:, :, 0] * data_moments[:, :, 0]
  data_moments[:, :, 4] = data_moments[:, :, 1] * data_moments[:, :, 1]
  data_moments[:, :, 5] = data_moments[:, :, 2] * data_moments[:, :, 2]
  data_moments[:, :, 6] = data_moments[:, :, 0] * data_moments[:, :, 1]
  data_moments[:, :, 7] = data_moments[:, :, 0] * data_moments[:, :, 2]
  data_moments[:, :, 8] = data_moments[:, :, 1] * data_moments[:, :, 2]

  

  return data_moments

# Add consistently oriented vertex normals

## Classic **PointNet**

## Classic **Momenet**

## $1^{st}$, $2^{nd}$ and $3^{rd}$ order moments

# Add another geometric prelifting

## Basic runs

## Consistently oriented vertex normals runs